In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir(r"/content/drive/MyDrive/ImageClef-2019-VQA-Med-Full")
os. getcwd()


In [ ]:
import os
import glob
import time
import datetime
import random

import numpy as np
import pandas as pd

from tqdm.notebook import tqdm

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup

import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

In [ ]:
if torch.cuda.is_available():


    device = torch.device("cuda")
    print('GPU:', torch.cuda.get_device_name(0))

else:
    print('Sem GPU disponivel')
    device = torch.device("cpu")

GPU: Tesla T4


In [ ]:
label_dict = {'C4_Abnormality':3,
             'C3_Organ':2,
             'C2_Plane':1,
             'C1_Modality':0}

In [ ]:
path =r'ImageClef-2019-VQA-Med-Training/ImageClef-2019-VQA-Med-Training/QAPairsByCategory'

filenames = glob.glob(path + "/*.txt")
dfs = []

for filename in filenames:
    df_temp = pd.read_csv(filename,delimiter='|',names=['id','question','answer'])
    df_temp['label'] = df_temp.apply(lambda x: label_dict.get((filename.split('/')[-1].replace('_train.txt',''))), axis=1)
    dfs.append(df_temp)

df_train = pd.concat(dfs, ignore_index=True)
df_train.drop(['id','answer'], axis = 1, inplace = True)



In [ ]:
path =r'ImageClef-2019-VQA-Med-Validation/ImageClef-2019-VQA-Med-Validation/QAPairsByCategory'

filenames = glob.glob(path + "/*.txt")

dfs = []

for filename in filenames:
    df_temp = pd.read_csv(filename,delimiter='|',names=['id','question','answer'])
    df_temp['label'] = df_temp.apply(lambda x: label_dict.get((filename.split('/')[-1].replace('_val.txt',''))), axis=1)
    dfs.append(df_temp)

df_valid = pd.concat(dfs, ignore_index=True)
df_valid.drop(['id','answer'], axis = 1, inplace = True)

In [ ]:
df_train.head()

,question,label
0,what kind of image is this?,0
1,is this a t1 weighted image?,0
2,what type of imaging modality is used to acqui...,0
3,is this a noncontrast mri?,0
4,what type of image modality is this?,0


In [ ]:
bert_model_name = 'bert-base-uncased'

In [ ]:
tokenizer = BertTokenizer.from_pretrained(bert_model_name, do_lower_case=True)
MAX_LEN = 128

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
def tokenize_sentences(sentences, tokenizer, max_seq_len = 128):
    input_ids = []
    attention_masks = []

    for sentence in tqdm(sentences):
        encoded_dict = tokenizer.encode_plus(
                        sentence,
                        add_special_tokens = True,
                        max_length = max_seq_len,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )

        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    return input_ids,attention_masks

In [ ]:
input_ids_train,attention_mask_train = tokenize_sentences(df_train['question'], tokenizer, MAX_LEN)
input_ids_valid,attention_mask_valid = tokenize_sentences(df_valid['question'], tokenizer, MAX_LEN)

  0%|          | 0/12792 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2699: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


  0%|          | 0/2000 [00:00<?, ?it/s]

In [ ]:
input_ids_train = torch.cat(input_ids_train, dim=0)
attention_mask_train = torch.cat(attention_mask_train, dim=0)
labels_train = torch.tensor(df_train['label'].values)

input_ids_valid = torch.cat(input_ids_valid, dim=0)
attention_mask_valid = torch.cat(attention_mask_valid, dim=0)
labels_test = torch.tensor(df_valid['label'].values)

train_dataset = TensorDataset(input_ids_train, attention_mask_train,labels_train )
val_dataset = TensorDataset(input_ids_valid, attention_mask_valid, labels_test)

In [ ]:
batch_size = 32


train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size
        )


validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset),
            batch_size = batch_size
        )

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 4,

    output_attentions = False,
    output_hidden_states = False,
)

model.cuda()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8
                )

epochs = 2


total_steps = len(train_dataloader) * epochs


scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# Add necessary imports
import random
import time
import numpy as np
import torch

def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss,
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode.
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 100 batches.
        if step % 100 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)

            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader.
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Clear any previously calculated gradients.
        model.zero_grad()

        # Perform a forward pass.
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask,
                        labels=b_labels)

        # Debugging: Check the type of `outputs`
        print(f"Type of outputs: {type(outputs)}")

        # Assuming outputs is a tuple of (loss, logits)
        loss = outputs[0]
        logits = outputs[1]

        # Debugging: Check the type of `loss`
        print(f"Type of loss: {type(loss)}")

        # Ensure `loss` is a tensor before calling `.item()`
        if isinstance(loss, torch.Tensor):
            total_train_loss += loss.item()
        else:
            raise ValueError(f"Expected loss to be a tensor but got {type(loss)}")

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))

    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode.
    model.eval()

    # Tracking variables
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:

        # Unpack this training batch from our dataloader.
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Forward pass, calculate logit predictions.
        with torch.no_grad():
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask,
                            labels=b_labels)

            # Debugging: Check the type of `outputs`
            print(f"Type of outputs during validation: {type(outputs)}")

            # Assuming outputs is a tuple of (loss, logits)
            loss = outputs[0]
            logits = outputs[1]

            # Debugging: Check the type of `loss`
            print(f"Type of loss during validation: {type(loss)}")

        # Ensure `loss` is a tensor before calling `.item()`
        if isinstance(loss, torch.Tensor):
            total_eval_loss += loss.item()
        else:
            raise ValueError(f"Expected validation loss to be a tensor but got {type(loss)}")

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences.
        total_eval_accuracy += flat_accuracy(logits, label_ids)

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)

    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))



======== Epoch 1 / 2 ========
Training...
Type of outputs: <class 'transformers.modeling_outputs.SequenceClassifierOutput'>
Type of loss: <class 'torch.Tensor'>
Type of outputs: <class 'transformers.modeling_outputs.SequenceClassifierOutput'>
Type of loss: <class 'torch.Tensor'>
Type of outputs: <class 'transformers.modeling_outputs.SequenceClassifierOutput'>
Type of loss: <class 'torch.Tensor'>
Type of outputs: <class 'transformers.modeling_outputs.SequenceClassifierOutput'>
Type of loss: <class 'torch.Tensor'>
Type of outputs: <class 'transformers.modeling_outputs.SequenceClassifierOutput'>
Type of loss: <class 'torch.Tensor'>
Type of outputs: <class 'transformers.modeling_outputs.SequenceClassifierOutput'>
Type of loss: <class 'torch.Tensor'>
Type of outputs: <class 'transformers.modeling_outputs.SequenceClassifierOutput'>
Type of loss: <class 'torch.Tensor'>
Type of outputs: <class 'transformers.modeling_outputs.SequenceClassifierOutput'>
Type of loss: <class 'torch.Tensor'>
Type 

In [ ]:
# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = './model_save/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))


Saving model to ./model_save/


('./model_save/tokenizer_config.json',
 './model_save/special_tokens_map.json',
 './model_save/vocab.txt',
 './model_save/added_tokens.json')